In [1]:
# Imports
import numpy as np
import csv
from collections import Counter
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from IPython.display import Image
from subprocess import call

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from keras.wrappers.scikit_learn import KerasClassifier

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras import backend as K

Using TensorFlow backend.
/Users/hanae/opt/anaconda3/envs/default/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hanae/opt/anaconda3/envs/default/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hanae/opt/anaconda3/envs/default/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/User

# Load data

In [2]:
train = pd.read_csv(os.path.join("cs5228", "train.csv"))
X_train = train.drop(["exceeds50K"], axis=1)
y_train = train["exceeds50K"]
X_test = pd.read_csv(os.path.join("cs5228", "test.csv"))
print(list(X_train.columns))
print(X_train.shape, X_test.shape)

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
(24421, 13) (24421, 13)


# Preprocessing

In [3]:
### Countries per region
new_countries = {
    "United-States": [' United-States'],
    "South-East-Asia": [' Philippines', ' Cambodia' ,' Laos', ' Vietnam', ' Thailand'],
    "North-America": [' Canada'],
    "Mexico": [' Mexico'],
    "Central-South-America": [' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador',
                              ' El-Salvador', ' Guatemala', ' Honduras', ' Jamaica', ' Peru',
                              ' Puerto-Rico', ' Trinadad&Tobago', ' Outlying-US(Guam-USVI-etc)',
                              ' Haiti', ' Nicaragua'],
    "Western-Europe": [' England', ' France', ' Germany', ' Holand-Netherlands',
                       ' Ireland', ' Scotland', ' Italy', ' Portugal'],
    "Eastern-Europe": [' Hungary', ' Greece', ' Yugoslavia', ' Poland'],
    "Asia": [' China', ' Hong', ' Japan', ' Taiwan'],
    "India": [' India'],
    "Iran": [' Iran'],
    "South": [' South'],
}

for k, v in new_countries.items():

    X_train['native-country'] = X_train['native-country'].replace(v, k)

    X_test['native-country'] = X_test['native-country'].replace(v, k)

### Log fnlwgt
X_train['fnlwgt'] = np.log(X_train['fnlwgt'])

X_test['fnlwgt'] = np.log(X_test['fnlwgt'])

### Log capital gain and loss
X_train['capital-gain'] = np.log(X_train['capital-gain']).replace(-np.inf, 0)

X_train['capital-loss'] = np.log(X_train['capital-loss']).replace(-np.inf, 0)

X_test['capital-gain'] = np.log(X_test['capital-gain']).replace(-np.inf, 0)

X_test['capital-loss'] = np.log(X_test['capital-loss']).replace(-np.inf, 0)

### Drop education, keep only education-num
X_train = X_train.drop(['education'], axis=1)

X_test = X_test.drop(['education'], axis=1)

X_train['relationship'] = X_train['relationship'].replace(to_replace =[" Wife", " Husband"], value='Spouse')

X_test['relationship'] = X_test['relationship'].replace(to_replace =[" Wife", " Husband"], value='Spouse')

### One-hot-encoding
X_train = pd.get_dummies(X_train)

X_test = pd.get_dummies(X_test)

/Users/hanae/opt/anaconda3/envs/default/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Model Keras

In [5]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def create_model(dense_layer_sizes=[64, 64], optimizer="adam", dropout=0.1, init='uniform', nbr_features=2500):
    model = Sequential()
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size, activation='relu', input_shape=(nbr_features,), kernel_initializer=init,)) 
        model.add(BatchNormalization())
        model.add(Dropout(dropout), )
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=[f1_m])
    return model

In [6]:
keras_estimator = KerasClassifier(build_fn=create_model, nbr_features=Xtrain.shape[1], verbose=1)

# fit the model
history = keras_estimator.fit(Xtrain, ytrain, validation_split=0.3, epochs=1000, verbose=1)

# evaluate the model
loss, f1_score = keras_estimator.evaluate(Xtest, ytest, verbose=0)

# Plot training & validation accuracy values
plt.plot(history.history['f1_m'])
plt.plot(history.history['val_f1_m'])
plt.title('Model accuracy')
plt.ylabel('F1 score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 11453 samples, validate on 4909 samples
Epoch 1/1000
11453/11453 [==============================] - 2s 135us/step - loss: 0.4241 - f1_m: 0.6141 - val_loss: 0.3559 - val_f1_m: 0.6544
Epoch 2/1000
11453/11453 [==============================] - 1s 82us/step - loss: 0.3558 - f1_m: 0.6330 - val_loss: 0.3379 - val_f1_m: 0.6582
Epoch 3/1000
11453/11453 [==============================] - 1s 83us/step - loss: 0.3547 - f1_m: 0.6261 - val_loss: 0.3995 - val_f1_m: 0.6708
Epoch 4/1000
11453/11453 [==============================] - 1s 94us/step - loss: 0.3488 - f1_m: 0.6323 - val_loss: 0.3251 - val_f1_m: 0.6274
Epoch 5/1000
11453/11453 [==============================] - 1s 83us/step - loss: 0.3433 - f1_m: 0.6341 - val_loss: 0.3597 - val_f1_m: 0.6641
Epoch 6/1000
11453/11453 [==============================] - 1s 83us/step - loss: 0.3439 - f1_m: 0.6317 - val_loss: 0.3355 - v

11453/11453 [==============================] - 1s 100us/step - loss: 0.3234 - f1_m: 0.6647 - val_loss: 0.3284 - val_f1_m: 0.6383
Epoch 55/1000
11453/11453 [==============================] - 1s 86us/step - loss: 0.3230 - f1_m: 0.6507 - val_loss: 0.3349 - val_f1_m: 0.6134
Epoch 56/1000
11453/11453 [==============================] - 1s 87us/step - loss: 0.3222 - f1_m: 0.6515 - val_loss: 0.3249 - val_f1_m: 0.6067
Epoch 57/1000
11453/11453 [==============================] - 1s 122us/step - loss: 0.3227 - f1_m: 0.6610 - val_loss: 0.3310 - val_f1_m: 0.5927
Epoch 58/1000
11453/11453 [==============================] - 1s 114us/step - loss: 0.3181 - f1_m: 0.6673 - val_loss: 0.3227 - val_f1_m: 0.6656
Epoch 59/1000
11453/11453 [==============================] - 1s 97us/step - loss: 0.3197 - f1_m: 0.6609 - val_loss: 0.3221 - val_f1_m: 0.6195
Epoch 60/1000
11453/11453 [==============================] - 1s 102us/step - loss: 0.3200 - f1_m: 0.6598 - val_loss: 0.3201 - val_f1_m: 0.6187
Epoch 61/1000
11

11453/11453 [==============================] - 1s 114us/step - loss: 0.3184 - f1_m: 0.6629 - val_loss: 0.3248 - val_f1_m: 0.6631
Epoch 169/1000
11453/11453 [==============================] - 1s 108us/step - loss: 0.3175 - f1_m: 0.6577 - val_loss: 0.3243 - val_f1_m: 0.6717
Epoch 170/1000
11453/11453 [==============================] - 1s 117us/step - loss: 0.3181 - f1_m: 0.6590 - val_loss: 0.3303 - val_f1_m: 0.5930
Epoch 171/1000
11453/11453 [==============================] - 1s 107us/step - loss: 0.3194 - f1_m: 0.6604 - val_loss: 0.3189 - val_f1_m: 0.6328
Epoch 172/1000
11453/11453 [==============================] - 1s 95us/step - loss: 0.3139 - f1_m: 0.6643 - val_loss: 0.3516 - val_f1_m: 0.6795
Epoch 173/1000
11453/11453 [==============================] - 1s 102us/step - loss: 0.3186 - f1_m: 0.6610 - val_loss: 0.3244 - val_f1_m: 0.6326
Epoch 174/1000
11453/11453 [==============================] - 1s 95us/step - loss: 0.3189 - f1_m: 0.6625 - val_loss: 0.3347 - val_f1_m: 0.6339
Epoch 175

11453/11453 [==============================] - 1s 97us/step - loss: 0.3113 - f1_m: 0.6727 - val_loss: 0.3247 - val_f1_m: 0.6086
Epoch 283/1000
11453/11453 [==============================] - 1s 98us/step - loss: 0.3121 - f1_m: 0.6706 - val_loss: 0.3217 - val_f1_m: 0.6133
Epoch 284/1000
11453/11453 [==============================] - 1s 97us/step - loss: 0.3147 - f1_m: 0.6581 - val_loss: 0.3207 - val_f1_m: 0.6453
Epoch 285/1000
11453/11453 [==============================] - 1s 90us/step - loss: 0.3167 - f1_m: 0.6686 - val_loss: 0.3272 - val_f1_m: 0.6696
Epoch 286/1000
11453/11453 [==============================] - 1s 89us/step - loss: 0.3153 - f1_m: 0.6620 - val_loss: 0.3216 - val_f1_m: 0.6186
Epoch 287/1000
11453/11453 [==============================] - 1s 92us/step - loss: 0.3113 - f1_m: 0.6711 - val_loss: 0.3214 - val_f1_m: 0.6236
Epoch 288/1000
11453/11453 [==============================] - 1s 101us/step - loss: 0.3136 - f1_m: 0.6684 - val_loss: 0.3268 - val_f1_m: 0.6029
Epoch 289/100

11453/11453 [==============================] - 2s 138us/step - loss: 0.3111 - f1_m: 0.6763 - val_loss: 0.3339 - val_f1_m: 0.5626
Epoch 397/1000
11453/11453 [==============================] - 1s 127us/step - loss: 0.3126 - f1_m: 0.6731 - val_loss: 0.3207 - val_f1_m: 0.6360
Epoch 398/1000
11453/11453 [==============================] - 2s 137us/step - loss: 0.3103 - f1_m: 0.6639 - val_loss: 0.3259 - val_f1_m: 0.6021
Epoch 399/1000
11453/11453 [==============================] - 1s 124us/step - loss: 0.3113 - f1_m: 0.6796 - val_loss: 0.3233 - val_f1_m: 0.6235
Epoch 400/1000
11453/11453 [==============================] - 1s 108us/step - loss: 0.3120 - f1_m: 0.6674 - val_loss: 0.3254 - val_f1_m: 0.6112
Epoch 401/1000
11453/11453 [==============================] - 1s 106us/step - loss: 0.3098 - f1_m: 0.6631 - val_loss: 0.3220 - val_f1_m: 0.6446
Epoch 402/1000
11453/11453 [==============================] - 2s 138us/step - loss: 0.3119 - f1_m: 0.6715 - val_loss: 0.3217 - val_f1_m: 0.6204
Epoch 4

...

11453/11453 [==============================] - 1s 89us/step - loss: 0.3079 - f1_m: 0.6723 - val_loss: 0.3211 - val_f1_m: 0.6344
Epoch 625/1000
11453/11453 [==============================] - 1s 92us/step - loss: 0.3059 - f1_m: 0.6909 - val_loss: 0.3242 - val_f1_m: 0.6293
Epoch 626/1000
11453/11453 [==============================] - 1s 92us/step - loss: 0.3091 - f1_m: 0.6791 - val_loss: 0.3206 - val_f1_m: 0.6366
Epoch 627/1000
11453/11453 [==============================] - 1s 93us/step - loss: 0.3030 - f1_m: 0.6820 - val_loss: 0.3224 - val_f1_m: 0.6476
Epoch 628/1000
11453/11453 [==============================] - 1s 96us/step - loss: 0.3128 - f1_m: 0.6640 - val_loss: 0.3248 - val_f1_m: 0.6158
Epoch 629/1000
11453/11453 [==============================] - 2s 136us/step - loss: 0.3116 - f1_m: 0.6734 - val_loss: 0.3201 - val_f1_m: 0.6084
Epoch 630/1000
11453/11453 [==============================] - 1s 124us/step - loss: 0.3036 - f1_m: 0.6724 - val_loss: 0.3298 - val_f1_m: 0.6676
Epoch 631/10

11453/11453 [==============================] - 1s 100us/step - loss: 0.3074 - f1_m: 0.6702 - val_loss: 0.3235 - val_f1_m: 0.6110
Epoch 739/1000
11453/11453 [==============================] - 1s 105us/step - loss: 0.3026 - f1_m: 0.6809 - val_loss: 0.3232 - val_f1_m: 0.6186
Epoch 740/1000
11453/11453 [==============================] - 1s 93us/step - loss: 0.3054 - f1_m: 0.6764 - val_loss: 0.3220 - val_f1_m: 0.6388
Epoch 741/1000
11453/11453 [==============================] - 1s 94us/step - loss: 0.3044 - f1_m: 0.6829 - val_loss: 0.3206 - val_f1_m: 0.6255
Epoch 742/1000
11453/11453 [==============================] - 1s 99us/step - loss: 0.3045 - f1_m: 0.6800 - val_loss: 0.3231 - val_f1_m: 0.6521
Epoch 743/1000
11453/11453 [==============================] - 1s 98us/step - loss: 0.3070 - f1_m: 0.6812 - val_loss: 0.3245 - val_f1_m: 0.6318
Epoch 744/1000
11453/11453 [==============================] - 1s 101us/step - loss: 0.3061 - f1_m: 0.6832 - val_loss: 0.3219 - val_f1_m: 0.6259
Epoch 745/1

11453/11453 [==============================] - 1s 98us/step - loss: 0.3060 - f1_m: 0.6777 - val_loss: 0.3255 - val_f1_m: 0.6482
Epoch 853/1000
11453/11453 [==============================] - 1s 87us/step - loss: 0.3030 - f1_m: 0.6833 - val_loss: 0.3289 - val_f1_m: 0.6129
Epoch 854/1000
11453/11453 [==============================] - 1s 96us/step - loss: 0.3032 - f1_m: 0.6836 - val_loss: 0.3214 - val_f1_m: 0.6456
Epoch 855/1000
11453/11453 [==============================] - 1s 91us/step - loss: 0.3031 - f1_m: 0.6716 - val_loss: 0.3219 - val_f1_m: 0.6505
Epoch 856/1000
11453/11453 [==============================] - 1s 98us/step - loss: 0.3058 - f1_m: 0.6844 - val_loss: 0.3237 - val_f1_m: 0.6334
Epoch 857/1000
11453/11453 [==============================] - 1s 101us/step - loss: 0.3023 - f1_m: 0.6812 - val_loss: 0.3253 - val_f1_m: 0.6346
Epoch 858/1000
11453/11453 [==============================] - 1s 99us/step - loss: 0.3038 - f1_m: 0.6833 - val_loss: 0.3250 - val_f1_m: 0.6322
Epoch 859/100

11453/11453 [==============================] - 1s 105us/step - loss: 0.3053 - f1_m: 0.6786 - val_loss: 0.3239 - val_f1_m: 0.6385
Epoch 967/1000
11453/11453 [==============================] - 1s 122us/step - loss: 0.3033 - f1_m: 0.6711 - val_loss: 0.3308 - val_f1_m: 0.5981
Epoch 968/1000
11453/11453 [==============================] - 1s 108us/step - loss: 0.3079 - f1_m: 0.6692 - val_loss: 0.3208 - val_f1_m: 0.6239
Epoch 969/1000
11453/11453 [==============================] - 1s 104us/step - loss: 0.3020 - f1_m: 0.6842 - val_loss: 0.3199 - val_f1_m: 0.6236
Epoch 970/1000
11453/11453 [==============================] - 1s 95us/step - loss: 0.3068 - f1_m: 0.6842 - val_loss: 0.3223 - val_f1_m: 0.6232
Epoch 971/1000
11453/11453 [==============================] - 1s 91us/step - loss: 0.3058 - f1_m: 0.6778 - val_loss: 0.3247 - val_f1_m: 0.6163
Epoch 972/1000
11453/11453 [==============================] - 1s 90us/step - loss: 0.3034 - f1_m: 0.6730 - val_loss: 0.3229 - val_f1_m: 0.6458
Epoch 973/

NameError: name 'model' is not defined

In [ ]:
dense_size_candidates = [[32], [64], [32, 32], [64, 64]]
my_classifier = KerasClassifier(make_model, batch_size=32)
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [3, 6],
                                     'filters': [8],
                                     'kernel_size': [3],
                                     'pool_size': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1)
validator.fit(x_train, y_train)

print('The parameters of the best model are: ')
print(validator.best_params_)

# Parameter optimisation in Sklearn: gridsearch using 5-fold CV for evaluation

In [11]:
## Create a parameter grid for gridsearch

random_grid = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,), (64,64,64), (64,128,64), (64,),
                          (64,128,128,64), (64,64,64,64), (64,64), (16,16,16), (16,32,16), (32,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

print(random_grid)

{'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50), (100,), (64, 64, 64), (64, 128, 64), (64,), (64, 128, 128, 64), (64, 64, 64, 64), (64, 64), (16, 16, 16), (16, 32, 16), (32,)], 'activation': ['tanh', 'relu'], 'solver': ['sgd', 'adam'], 'alpha': [0.0001, 0.05], 'learning_rate': ['constant', 'adaptive']}


In [12]:
## Use the random grid to search for best hyperparameters

# First create the base model to tune
mlp = MLPClassifier()
# Random search of parameters, using 3 fold cross validation, search across n_iter different combinations, and use all available cores
mlp_gridsearch = RandomizedSearchCV(estimator=mlp, param_distributions=random_grid,
                                   scoring="f1_weighted", n_iter=2000, cv=3, verbose=6, n_jobs=4, random_state=42)
# Fit the random searfch model
mlp_gridsearch.fit(X_train, y_train)
# Print best parameters according to gridsearch
mlp_gridsearch.best_params_

Fitting 3 folds for each of 192 candidates, totalling 576 fits


/Users/hanae/opt/anaconda3/envs/default/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 192 is smaller than n_iter=2000. Running 192 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  7.4min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed: 23.2min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 33.2min
[Parallel(n_jobs=4)]: Done 305 tasks      | elapsed: 50.9min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 77.6min
[Parallel(n_jobs=4)]: Done 576 out of 576 | elapsed: 98.3min finished


{'solver': 'adam',
 'learning_rate': 'adaptive',
 'hidden_layer_sizes': (100,),
 'alpha': 0.05,
 'activation': 'tanh'}

In [14]:
## Evaluate improvment between the base model and the fine-tuned model

base_model = MLPClassifier(random_state=42)
cv_score_base = cross_val_score(base_model, X_train, y_train, cv=5, verbose=6, scoring='f1_macro')
print(cv_score_base.mean(), cv_score_base.std())

# best_model = rf_gridsearch.best_estimator_
best_params = {'solver': 'adam',
 'learning_rate': 'adaptive',
 'hidden_layer_sizes': (100,),
 'alpha': 0.05,
 'activation': 'tanh'}

best_model = MLPClassifier(**best_params, random_state=42)
cv_score_best = cross_val_score(best_model, X_train, y_train, cv=5, verbose=6, scoring='f1_macro')
print(cv_score_best.mean(), cv_score_best.std())

print('Improvement of {:0.2f}%.'.format(100 * (cv_score_best.mean() - cv_score_base.mean()) / cv_score_base.mean()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] .................................... , score=0.738, total=  18.8s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.8s remaining:    0.0s


[CV] .................................... , score=0.808, total=  17.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   36.0s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   33.7s remaining:    0.0s


[CV] .................................... , score=0.781, total=  12.9s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   46.6s remaining:    0.0s


[CV] .................................... , score=0.794, total=  10.4s
0.7885324098635764 0.013141062110172606
Improvement of 0.28%.


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   57.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   57.0s finished
